In [2]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd

In [7]:
benchmark = 'megadepth1500'
# benchmark = 'graz_high_res'

with open(f'{benchmark}/results/results.json') as f:
    data = json.load(f)

df = pd.DataFrame(data).T.round(3)
df['inlier'] = round(df['inlier']).astype(int)
df['auc_5'] *= 100
df['auc_10'] *= 100
df['model'] = df.index.str.split(' ').str[0]
df['params'] = df.index.str.split(' ').str[1]
df['scale'] = df.index.str.split(' ').str[2]
df['timestamp'] = df.index.str.split(' ').str[-1]

df = df[['model',  'scale', 'inlier', 'auc_5', 'auc_10', 'timestamp']]
df.sort_values(by=['scale', 'model'], inplace=True, ascending=[True, True])
df.index = range(1, 1+len(df.index))
df

,model,scale,inlier,auc_5,auc_10,timestamp
1,aliked,2048,456,40.7,56.6,20250901_230158
2,aliked+SANDesc,2048,477,43.5,60.1,20250901_230721
3,dedode+SANDesc,2048,272,45.2,61.8,20250902_092140
4,dedode-B,2048,240,40.5,56.6,20250902_000302
5,dedode-G,2048,265,44.6,61.8,20250902_001933
6,disk,2048,457,34.7,51.4,20250901_231502
7,disk+SANDesc,2048,477,36.6,54.2,20250901_232106
8,ripe,2048,335,42.3,58.0,20250901_234255
9,ripe+SANDesc,2048,325,42.9,58.7,20250901_235149
10,superpoint,2048,210,28.9,44.9,20250901_232941
